In [1]:
import pandas as pd
import numpy as np
import regex as rg

In [2]:
folder = 'C:\\Users\\jessi\\Documents\\TCC Docs\\Datasets Covid\\'

In [3]:
#Dados Fleury
fleury_pacientes = pd.read_csv(folder + "GrupoFleuryAgosto\\GrupoFleury_Pacientes_2.csv", sep = "|", encoding="utf-8")
fleury_exames = pd.read_csv(folder + "GrupoFleuryAgosto\\GrupoFleury_Exames_2.csv", sep = "|", encoding="utf-8")
fleury_exames['DE_EXAME'] = fleury_exames['DE_EXAME'] + " - " + fleury_exames['DE_ANALITO']

In [4]:
df = pd.DataFrame()
df['EXAME'] = fleury_exames['DE_EXAME'] + ' - ' + fleury_exames['DE_ANALITO']
df['UNIDADE'] = fleury_exames['CD_UNIDADE']
df['VALOR_REF'] = fleury_exames['DE_VALOR_REFERENCIA']
technical_fleury = df.drop_duplicates()
technical_fleury.to_csv('Fleury_Exames.csv', index=False, sep="|",encoding='utf-8-sig')

In [5]:
cols_exam = fleury_exames['DE_EXAME'].value_counts().to_frame(name='Preenchidos')
cols_exam['%'] = (cols_exam['Preenchidos'] / fleury_exames.shape[0]) * 100
to_drop = cols_exam[cols_exam['Preenchidos'] < 10000].index.to_list()
fleury_exames = fleury_exames[~fleury_exames['DE_EXAME'].isin(to_drop)]
fleury_exames['DE_EXAME'].value_counts()

COVID19, ANTICORPOS IgG, soro - Covid 19, Anticorpos IgG, Quimiolumin.-Índice        181532
COVID19, ANTICORPOS IgM, soro - Covid 19, Anticorpos IgM, Quimiolumin.-Índice        181506
COVID19, ANTICORPOS IgG, soro - Covid 19, Anticorpos IgG, Quimioluminescência        180301
COVID19, ANTICORPOS IgM, soro - Covid 19, Anticorpos IgM, Quimioluminescência        180272
NOVO CORONAVÍRUS 2019 (SARS-CoV-2), DETECÇÃO POR PCR - Covid 19, Detecção por PCR     95534
                                                                                      ...  
ANTIGENO PROSTATICO ESPECIFICO(PSA), soro - PSA Livre                                 10464
DESIDROGENASE LACTICA (DHL), soro - DHL                                               10181
HOMOCISTEINA, plasma - Homocisteína                                                   10103
ANTIGENO CARCINOEMBRIONICO (CEA), soro - CEA                                          10085
SATURAÇÃO DA TRANSFERRINA, soro - Ferro - Grau de Saturação                     

In [6]:
fleury_exames['DT_COLETA'] = pd.to_datetime(fleury_exames['DT_COLETA'], format='%d/%m/%Y')
fleury_exames['DE_RESULTADO'].replace('','*',inplace=True)
to_drop = ['*', 'NOVA COLETA', 'Nova coleta', 'nova coleta', 'Nova Coleta', 'Indeterminado', 'Inconclusivo']
fleury_exames = fleury_exames[~fleury_exames['DE_RESULTADO'].isin(to_drop)]
del fleury_exames['DE_ANALITO']
fleury_exames.sort_values(['ID_PACIENTE','DT_COLETA', 'DE_EXAME'], ascending=[False, False, False],inplace=True)

In [7]:
fleury_exames.sort_values(by='DT_COLETA',ascending=False)

,ID_PACIENTE,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
2210584,BC1D313F80F0B0C90F1D0217E6D4C03A,2020-07-31,LAB,"HEMOGRAMA, sangue total - RDW","12,3",%,"11,9 a 15,5"
2753766,655A209D13AEBBE4D11BBC130BCE1748,2020-07-31,LAB,"URINA TIPO I - JATO MEDIO - Glicose, urina iso...","inferior a 0,3 g/L",g/L,"inferior a 0,3"
5866694,7CA36AE4B4C0A38605CC8A2C02CF3906,2020-07-31,LAB,"COVID19, ANTICORPOS IgG, soro - Covid 19, Anti...",NÃO REAGENTE,NaN,Não reagente
5866695,7CA36AE4B4C0A38605CC8A2C02CF3906,2020-07-31,LAB,"COVID19, ANTICORPOS IgM, soro - Covid 19, Anti...","0,9",UA/mL,"inferior a 0,8"
5866697,7CA36AE4B4C0A38605CC8A2C02CF3906,2020-07-31,LAB,"COVID19, ANTICORPOS IgM, soro - Covid 19, Soro...",Ausência de anticorpos contra SARS-CoV-2 detec...,NaN,NaN
...,...,...,...,...,...,...,...
2983439,CD56AA4D2C3D01D1BEEE1A3F8686039F,2019-11-01,LAB,"HEMOGRAMA, sangue total - Hemoglobina Corpuscu...","30,5",pg,"26,0 a 34,0"
2983430,CD56AA4D2C3D01D1BEEE1A3F8686039F,2019-11-01,LAB,"HEMOGRAMA, sangue total - Leucócitos",6530,/mm3,3.500 a 10.500
2983434,CD56AA4D2C3D01D1BEEE1A3F8686039F,2019-11-01,LAB,"HEMOGRAMA, sangue total - Linfócitos",2650,/mm3,900 a 2.900
2983445,CD56AA4D2C3D01D1BEEE1A3F8686039F,2019-11-01,LAB,"HEMOGRAMA, sangue total - Linfócitos (%)","40,6",%,NaN


In [7]:
fleury_exames_exp = fleury_exames.pivot_table(index= ['ID_PACIENTE','DT_COLETA'], columns='DE_EXAME', values='DE_RESULTADO', aggfunc=';'.join)

In [8]:
cols_fleury = pd.DataFrame(fleury_exames_exp.notna().sum(),columns=['Preenchidos'])
cols_fleury['%'] = (cols_fleury['Preenchidos'] / fleury_exames_exp.shape[0]) * 100
cols_fleury[cols_fleury.index.str.contains('BILIRRUBINAS')].sort_values(by='%', ascending=False)

,Preenchidos,%
DE_EXAME,,
"BILIRRUBINAS, soro - Bilirrubina Direta",24059,6.644756
"BILIRRUBINAS, soro - Bilirrubina Indireta",24059,6.644756
"BILIRRUBINAS, soro - Bilirrubina Total",24059,6.644756


In [11]:
cols_to_save = ['BILIRRUBINAS, soro - Bilirrubina Direta', 'BILIRRUBINAS, soro - Bilirrubina Indireta', 
                'BILIRRUBINAS, soro - Bilirrubina Total', 'CREATININA, soro - Creatinina',
                'CALCIO IONIZADO, soro - Cálcio Ionizado', 'FOSFATASE ALCALINA (ALP), soro - Fosfatase Alcalina', 
                'GAMA GLUTAMIL TRANSFERASE (GAMA-GT), soro - Gama-GT', 'GLICOSE, plasma - Glicose',
                'GLICOSE, soro - Glicose', 'MAGNESIO, soro - Magnésio', 'POTASSIO, soro - Potássio', 
                'POTASSIO, soro - Potássio', 'SODIO, soro - Sódio', 'TRANSAMINASE GLUTAMICO-OXALACETICA (TGO) - AST (TGO)',
                'TRANSAMINASE GLUTAMICO-PIRUVICA (TGP) - ALT (TGP)', 'UREIA, soro - Uréia',
                'HEMOGRAMA, sangue total - Basófilos (%)', 'HEMOGRAMA, sangue total - Basófilos',
                'HEMOGRAMA, sangue total - Eosinófilos (%)', 'HEMOGRAMA, sangue total - Eosinófilos', 
                'HEMOGRAMA, sangue total - Linfócitos (%)', 'HEMOGRAMA, sangue total - Linfócitos',
                'HEMOGRAMA, sangue total - Monócitos (%)', 'HEMOGRAMA, sangue total - Monócitos',
                'HEMOGRAMA, sangue total - Neutrófilos (%)', 'HEMOGRAMA, sangue total - Neutrófilos', 
                'HEMOGRAMA, sangue total - Concentração de Hemoglobina Corpuscular',
                'HEMOGRAMA, sangue total - Hemoglobina Corpuscular Média',
                'HEMOGRAMA, sangue total - Hematócrito', 'HEMOGRAMA, sangue total - Hemoglobina',
                'HEMOGRAMA, sangue total - Leucócitos', 'HEMOGRAMA, sangue total - Plaquetas',
                'HEMOGRAMA, sangue total - RDW', 'HEMOGRAMA, sangue total - Eritrócitos',
                'HEMOGRAMA, sangue total - Volume plaquetário médio',
                'NOVO CORONAVÍRUS 2019 (SARS-CoV-2), DETECÇÃO POR PCR - Covid 19, Detecção por PCR',
                'COVID19, ANTICORPOS IgM, soro - Covid 19, Anticorpos IgM, Quimioluminescência',
                'URINA TIPO I - JATO MEDIO - Aspecto, urina', 'URINA TIPO I - JATO MEDIO - Glicose, urina isolada',
                'URINA TIPO I - JATO MEDIO - Cilindros, urina', 'URINA TIPO I - JATO MEDIO - Cor, urina',
                'URINA TIPO I - JATO MEDIO - Corpos cetônicos, urina', 'URINA TIPO I - JATO MEDIO - Densidade, urina',
                'URINA TIPO I - JATO MEDIO - Leucócitos quantitativo, urina', 'URINA TIPO I - JATO MEDIO - Nitrito, urina',
                'URINA TIPO I - JATO MEDIO - Proteínas, urina isolada', 'URINA TIPO I - JATO MEDIO - Urobilinogênio, urina',
                'URINA TIPO I - JATO MEDIO - pH, urina']

In [12]:
df_fleury_exames = fleury_exames_exp[cols_to_save]

In [13]:
cols_hemograma = cols_fleury[cols_fleury.index.str.contains('HEMOGRAMA')].index
cols_urina = cols_fleury[cols_fleury.index.str.contains('URINA')].index
df_fleury_exames = df_fleury_exames[df_fleury_exames[cols_urina[0]].notna() == True]
df_fleury_exames =  df_fleury_exames[df_fleury_exames[cols_hemograma[0]].notna() == True]

In [15]:
df_fleury_exames['DIAGNOSTICO_COVID'] = 'PCR - ' + df_fleury_exames['NOVO CORONAVÍRUS 2019 (SARS-CoV-2), DETECÇÃO POR PCR - Covid 19, Detecção por PCR']
df_fleury_exames.loc[df_fleury_exames['DIAGNOSTICO_COVID'].isna() & \
                       df_fleury_exames['COVID19, ANTICORPOS IgM, soro - Covid 19, Anticorpos IgM, Quimioluminescência'].notna(),
                      'DIAGNOSTICO_COVID'] = 'Sorologia IgM - ' + df_fleury_exames['COVID19, ANTICORPOS IgM, soro - Covid 19, Anticorpos IgM, Quimioluminescência']

In [16]:
df_fleury_exames = df_fleury_exames[~df_fleury_exames['DIAGNOSTICO_COVID'].isna()]

In [17]:
df_fleury_exames['DIAGNOSTICO_COVID'].value_counts()

Sorologia IgM - NÃO REAGENTE                 3678
PCR - NÃO DETECTADO (NEGATIVO)                305
Sorologia IgM - REAGENTE                      269
PCR - NÃO DETECTADO                            98
PCR - DETECTADO (POSITIVO)                     35
PCR - DETECTADO                                20
Sorologia IgM - NÃO REAGENTE;NÃO REAGENTE       2
Name: DIAGNOSTICO_COVID, dtype: int64

In [18]:
df_fleury_exames.to_csv('Fleury_v2.csv',sep="|",encoding='utf-8-sig')